In [12]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import pandas as pd

# Data Processing helpers

In [13]:
data_path = 'C:/Users/ehniv/Desktop/MATH448-SaveOn/highgates_village_test_1'

#processes the time matrix
time_matrix = pd.read_csv(data_path+'/time_matrix.csv', index_col = 'Unnamed: 0')
processed_time_matrix = time_matrix.values.tolist()

In [14]:
time_slot_data = pd.read_csv(data_path+'/time_slot.csv', index_col = 'Unnamed: 0')

#creates a tuple of time data
processed_time_windows = time_slot_data[["start_time_scaled","end_time_scaled"]].apply(tuple, axis = 1).to_list()

In [15]:
print(processed_time_matrix)

[[0, 13, 20, 6, 6, 18, 17, 11, 6, 5, 16, 10, 19, 7, 10, 9, 10, 10, 8, 18, 10, 18], [14, 0, 9, 10, 10, 9, 7, 6, 11, 10, 7, 7, 9, 8, 5, 6, 7, 6, 7, 8, 6, 8], [20, 9, 0, 15, 15, 1, 3, 13, 17, 15, 5, 14, 3, 14, 12, 12, 13, 13, 13, 3, 12, 2], [7, 9, 15, 0, 4, 14, 13, 10, 4, 2, 12, 6, 15, 3, 7, 5, 9, 6, 4, 14, 7, 14], [8, 10, 15, 3, 0, 15, 13, 9, 5, 2, 13, 7, 15, 3, 7, 5, 9, 6, 4, 14, 7, 14], [19, 9, 1, 14, 14, 0, 3, 14, 16, 14, 5, 13, 3, 13, 11, 12, 14, 13, 12, 3, 11, 2], [17, 7, 4, 13, 13, 4, 0, 12, 14, 13, 2, 11, 5, 11, 10, 10, 12, 11, 10, 2, 9, 3], [13, 7, 13, 10, 9, 14, 12, 0, 11, 8, 11, 7, 14, 8, 5, 7, 1, 6, 6, 13, 6, 13], [7, 11, 17, 4, 5, 16, 14, 11, 0, 4, 14, 8, 16, 4, 8, 6, 11, 7, 5, 15, 8, 15], [6, 9, 15, 2, 2, 14, 13, 8, 4, 0, 13, 6, 15, 3, 7, 5, 8, 6, 4, 14, 7, 14], [17, 7, 5, 13, 12, 5, 2, 11, 14, 12, 0, 11, 5, 11, 9, 10, 11, 11, 10, 3, 9, 4], [11, 6, 13, 6, 6, 12, 10, 8, 8, 6, 10, 0, 13, 5, 3, 3, 9, 2, 3, 12, 2, 12], [20, 10, 3, 15, 15, 2, 3, 13, 17, 15, 5, 14, 0, 14, 12, 13, 

In [16]:
processed_time_windows

[(0, 420),
 (120, 240),
 (120, 240),
 (0, 420),
 (300, 420),
 (0, 420),
 (0, 420),
 (0, 420),
 (120, 240),
 (300, 420),
 (0, 420),
 (300, 420),
 (0, 420),
 (0, 120),
 (0, 420),
 (0, 120),
 (0, 120),
 (0, 120),
 (120, 240),
 (120, 240),
 (0, 120),
 (300, 420)]

# Config

In [17]:
#helper functions
def addServiceTime(time_matrix, service_time):
    for from_idx in range(1,len(time_matrix)):
        for to_idx in range(len(time_matrix[0])):
            if from_idx != to_idx:
                time_matrix[from_idx][to_idx] = service_time + time_matrix[from_idx][to_idx]

def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['time_matrix'] = processed_time_matrix
    data['time_windows'] = processed_time_windows
    data['num_vehicles'] = 1
    data['num_locations'] = len(data['time_windows']) - 1
    data['depot'] = 0 #depot location based on item
    
    #driver breaks
    data['driver_break_duration'] = 30
    data['driver_break_start'] = 240 # hours past 7AM * 60 minutes/hour
    data['driver_break_end'] = 360 
    data['breaks'] = [data['driver_break_duration'] for _ in range(data['num_vehicles'])]
    
    #service times
    data['service_time'] = 10
    data['maximum_wait_at_location'] = 10000
    addServiceTime(data['time_matrix'], data['service_time'])
    
    
    #vehicle capacities
    data['demands'] = [0] + [1 for _ in range(data['num_locations'])]
    data['max_vehicle_capacities'] = 21
    load_balancing = False;
    limit_vehicle_capacity = 0.8 #only applied if load_balancing = true
    
    if load_balancing == True:
        data['vehicle_capacities'] = [data['max_vehicle_capacities']*limit_vehhicle_capacity for _ in range(data['num_vehicles'])]
    else:
        data['vehicle_capacities'] = [data['max_vehicle_capacities'] for _ in range(data['num_vehicles'])]
    
    #solver config
    data['solution_limit'] = 20000 #40000 #upper bound of # of solutions to calculate
    data['drop_penalty'] = 99999
    

    return data


def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f'Objective: {solution.ObjectiveValue()}')
    
    #prints breaks
    print('Breaks:')
    intervals = solution.IntervalVarContainer()
    for i in range(intervals.Size()):
        brk = intervals.Element(i)
        if brk.PerformedValue() == 1:
            print('{}: Start({}) Duration({})'.format(
                brk.Var().Name(),
                brk.StartValue(),
                brk.DurationValue()))
        else:
            print('{}: Unperformed'.format(brk.Var().Name()))
    print()
    
    
    #Prints dropped nodes.
    dropped_nodes = 'Dropped nodes:'
    for node in range(routing.Size()):
        if routing.IsStart(node) or routing.IsEnd(node):
            continue
        if solution.Value(routing.NextVar(node)) == node:
            dropped_nodes += ' {}'.format(manager.IndexToNode(node))
    print(dropped_nodes, '\n')
    
    
    time_dimension = routing.GetDimensionOrDie('Time')
    total_time = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        
        isFirstNode = True
        while not routing.IsEnd(index):
            time_var = time_dimension.CumulVar(index)
            
            if isFirstNode:
            
                plan_output += '{0} Time({1},{2}) -> '.format(
                    manager.IndexToNode(index), solution.Min(time_var),
                    solution.Max(time_var))
                index = solution.Value(routing.NextVar(index))
                isFirstNode = False
            else:
                plan_output += '{0} Time({1},{2}) -> '.format(
                    manager.IndexToNode(index), solution.Min(time_var),
                    solution.Max(time_var) + data["service_time"])
                index = solution.Value(routing.NextVar(index))
            
            
            
        time_var = time_dimension.CumulVar(index)
        plan_output += '{0} Time({1},{2})\n'.format(manager.IndexToNode(index),
                                                    solution.Min(time_var),
                                                    solution.Max(time_var))
        plan_output += 'Time of the route: {}min\n'.format(
            solution.Min(time_var))
        print(plan_output)
        total_time += solution.Min(time_var)
    print('Total time of all routes: {}min'.format(total_time))

# TSP Solver

In [ ]:
"""Solve the VRP with time windows."""
# Instantiate the data problem.
data = create_data_model()

# Create the routing index manager.
manager = pywrapcp.RoutingIndexManager(len(data['time_matrix']),
                                       data['num_vehicles'], data['depot'])

#used for tracing/debugging
# routing_parameters = pywrapcp.DefaultRoutingModelParameters()
# routing_parameters.solver_parameters.trace_propagation = True
# routing_parameters.solver_parameters.trace_search = True

# Create Routing Model.
routing = pywrapcp.RoutingModel(manager) #, routing_parameters)


# Create and register a transit callback.
def time_callback(from_index, to_index):
    """Returns the travel time between the two nodes."""
    # Convert from routing variable Index to time matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data['time_matrix'][from_node][to_node]

transit_callback_index = routing.RegisterTransitCallback(time_callback)

# Define cost of each arc.
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)


# Adds Capacity Constraints
def demand_callback(from_index):
    """Returns the demand of the node."""
    # Convert from routing variable Index to demands NodeIndex.
    from_node = manager.IndexToNode(from_index)
    return data['demands'][from_node]

demand_callback_index = routing.RegisterUnaryTransitCallback(
    demand_callback)
routing.AddDimensionWithVehicleCapacity(
    demand_callback_index,
    10000000,  # null capacity slack
    data['vehicle_capacities'],  # vehicle maximum capacities
    True,  # start cumul to zero
    'Capacity')

# Allow to drop nodes.
penalty = data['drop_penalty']
for node in range(1, len(data['time_matrix'])):
    routing.AddDisjunction([manager.NodeToIndex(node)], penalty)

# Add Time Windows constraint.
time = 'Time'
routing.AddDimension(
    transit_callback_index,
    data['maximum_wait_at_location'],  # allow waiting time
    data['maximum_wait_at_location'],  # maximum time per vehicle
    False,  # Don't force start cumul to zero.
    time)
time_dimension = routing.GetDimensionOrDie(time)


# Add time window constraints for each location except depot.
for location_idx, time_window in enumerate(data['time_windows']):
    if location_idx == data['depot']:
        continue
    index = manager.NodeToIndex(location_idx)
    time_dimension.CumulVar(index).SetRange(time_window[0], time_window[1])
    
    
# Add time window constraints for each vehicle start node.
depot_idx = data['depot']
for vehicle_id in range(data['num_vehicles']):
    index = routing.Start(vehicle_id)
    time_dimension.CumulVar(index).SetRange(
        data['time_windows'][depot_idx][0],
        data['time_windows'][depot_idx][1])
    
    
# Add driver break constraints
node_visit_transit = {}
for n in range(routing.Size()):
    if n >= len(data['time_windows']):
        node_visit_transit[n] = 0
    else:
        node_visit_transit[n] = int(data['demands'][n] * data['service_time'])

break_intervals = {}
for v in range(data['num_vehicles']):
    vehicle_break = data['breaks'][v]
    break_intervals[v] = [
        routing.solver().FixedDurationIntervalVar(
            data['driver_break_start'], 
            data['driver_break_end'],
            data['driver_break_duration'],
            False,
            'Break for vehicle {}'.format(v))
    ]
    time_dimension.SetBreakIntervalsOfVehicle(
        break_intervals[v], v, node_visit_transit)

    

# Instantiate route start and end times to produce feasible times.
for i in range(data['num_vehicles']):
    routing.AddVariableMinimizedByFinalizer(
        time_dimension.CumulVar(routing.Start(i)))
    routing.AddVariableMinimizedByFinalizer(
        time_dimension.CumulVar(routing.End(i)))

# Setting first solution heuristic.
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.local_search_metaheuristic = (
   routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
search_parameters.log_search = True
search_parameters.solution_limit = data['solution_limit']


# Solve the problem.
solution = routing.SolveWithParameters(search_parameters)
# Print solution on console.
if solution:
    print_solution(data, manager, routing, solution)
else:
    print("no solutions found")

